In [1]:
!wget 'https://pages.mini.pw.edu.pl/~chilinskim/GO_files/IntermediateFiles/SRR_final_sorted.bam'

--2024-06-07 14:14:17--  https://pages.mini.pw.edu.pl/~chilinskim/GO_files/IntermediateFiles/SRR_final_sorted.bam
Resolving pages.mini.pw.edu.pl (pages.mini.pw.edu.pl)... 194.29.178.29
Connecting to pages.mini.pw.edu.pl (pages.mini.pw.edu.pl)|194.29.178.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622888160 (594M)
Saving to: ‘SRR_final_sorted.bam’

SRR_final_sorted.ba 100%[===================>] 594.03M  2.63MB/s    in 3m 58s  

2024-06-07 14:18:16 (2.50 MB/s) - ‘SRR_final_sorted.bam’ saved [622888160/622888160]



In [2]:
!pip install pysam tqdm cyvcf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.5 MB/s eta 0:00:00


In [3]:
import pysam
from tqdm.notebook import tqdm
import numpy as np
from cyvcf2 import VCF, Writer, Variant
from scipy import ndimage
import matplotlib.pyplot as plt

In [4]:
def calculate_read_depth(bam_file, binwidth):
    bam = pysam.AlignmentFile(bam_file, "rb")
    depth_dict = {}

    for pileupcolumn in tqdm(bam.pileup(), total=108148575):
        chrom = pileupcolumn.reference_name
        if chrom not in ALLOWED_CHR:
            continue
        bin_start = (pileupcolumn.pos // binwidth) * binwidth
        depth = pileupcolumn.nsegments

        if depth == 0:
            continue

        if chrom not in depth_dict:
            depth_dict[chrom] = {}

        if bin_start not in depth_dict[chrom]:
            depth_dict[chrom][bin_start] = 0

        depth_dict[chrom][bin_start] += depth

    bam.close()
    return depth_dict

def smooth_signal(depth_dict, sigma):
    smoothed_dict = {}

    for chrom in tqdm(depth_dict):
        bins = sorted(depth_dict[chrom].keys())
        depths = np.array([depth_dict[chrom][bin_start] for bin_start in bins])

        # smoothed_depths = np.convolve(depths, np.ones(window_size)/window_size, mode='same')
        smoothed_depths = ndimage.gaussian_filter1d(depths, sigma)

        smoothed_dict[chrom] = {bins[i]: smoothed_depths[i] for i in range(len(bins))}

    return smoothed_dict

def find_cutoff(signal):
    nonzero = signal[np.where(signal != 0)]  # filter zeros due to bad reads
    log_signal = np.log(nonzero)
    mean_log_signal = np.mean(log_signal)
    std_log_signal = np.std(log_signal)
    return np.exp(mean_log_signal - 2*std_log_signal), np.exp(mean_log_signal + 2*std_log_signal)


def detect_variants(depth_dict, binwidth, threshold=30):
    variants = []

    for chrom in tqdm(depth_dict, total=len(ALLOWED_CHR), position=0):
        bins = list(depth_dict[chrom].keys())
        depths = np.array([depth_dict[chrom][bin_start] for bin_start in bins])

        del_threshold, ins_threshold = find_cutoff(depths)
        tqdm.write(f"Chromosome {chrom} | DEL threshold: {del_threshold: .2f} | INS threshold: {ins_threshold: .2f}")
        for bin_start in bins:
            depth = depth_dict[chrom][bin_start]
            if depth < del_threshold:  # Potential deletion
                variants.append((chrom, bin_start, bin_start + binwidth, "DEL", depth))
            elif depth > ins_threshold:  # Potential insertion
                variants.append((chrom, bin_start, bin_start + binwidth, "INS", depth))

    return variants

def merge_variants(variants, binwidth):
    if not variants:
        return variants

    merged_variants = []
    prev_variant = variants[0]

    for current_variant in tqdm(variants[1:]):

        prev_chrom, prev_start, prev_end, prev_type, prev_depth = prev_variant
        curr_chrom, curr_start, curr_end, curr_type, curr_depth = current_variant

        if prev_chrom == curr_chrom and prev_type == curr_type and prev_end == curr_start:
            # Merge the variants
            # tqdm.write(f"Chromosome {prev_chrom} | Merged {prev_type} | Loc1: {prev_start} | Loc2: {curr_start}")
            prev_variant = (prev_chrom, prev_start, curr_end, prev_type, prev_depth + curr_depth)
        else:
            merged_variants.append(prev_variant)
            prev_variant = current_variant

    merged_variants.append(prev_variant)  # Append the last variant
    return merged_variants

def create_vcf(variants, output_file):
    vcf_template = """##fileformat=VCFv4.2
##source=readDepth
##reference=GRCh38
#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO
"""
    with open(output_file, 'w') as vcf:
        vcf.write(vcf_template)
        for chrom, start, end, var_type, depth in variants:
            ref = "."
            alt = "<{}>".format(var_type)
            info = "END={};DP={}".format(end, depth)
            vcf.write("{}\t{}\t.\t{}\t{}\t.\tPASS\t{}\n".format(chrom, start + 1, ref, alt, info))


In [5]:
bam_filename = 'SRR_final_sorted.bam'

In [6]:
bamfile = pysam.AlignmentFile(bam_filename, "rb")
pysam.index(bam_filename)
bamfile = pysam.AlignmentFile(bam_filename, "rb")

In [7]:
ALLOWED_CHR = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY']

In [8]:
bam_file = bam_filename
output_vcf = "detected_variants.vcf"
binwidth = 500  # Example bin width, adjust as needed

In [9]:
depth_dict = calculate_read_depth(bam_filename, binwidth)

  0%|          | 0/108148575 [00:00<?, ?it/s]

In [10]:
smoothed_depths = smooth_signal(depth_dict, 100)

  0%|          | 0/24 [00:00<?, ?it/s]

In [11]:
x = []
y = []
for k,v in smoothed_depths['chr1'].items():
    x.append(k)
    y.append(v)

In [12]:
variants = detect_variants(smoothed_depths, binwidth)

  0%|          | 0/24 [00:00<?, ?it/s]

Chromosome chr1 | DEL threshold:  183.45 | INS threshold:  1267.40
Chromosome chr2 | DEL threshold:  153.12 | INS threshold:  927.41
Chromosome chr3 | DEL threshold:  145.02 | INS threshold:  950.95
Chromosome chr4 | DEL threshold:  140.61 | INS threshold:  721.69
Chromosome chr5 | DEL threshold:  138.80 | INS threshold:  868.04
Chromosome chr6 | DEL threshold:  149.03 | INS threshold:  860.83
Chromosome chr7 | DEL threshold:  147.28 | INS threshold:  938.63
Chromosome chr8 | DEL threshold:  144.06 | INS threshold:  722.69
Chromosome chr9 | DEL threshold:  179.58 | INS threshold:  957.54
Chromosome chr10 | DEL threshold:  165.78 | INS threshold:  906.77
Chromosome chr11 | DEL threshold:  158.16 | INS threshold:  1232.23
Chromosome chr12 | DEL threshold:  166.39 | INS threshold:  1095.97
Chromosome chr13 | DEL threshold:  131.67 | INS threshold:  639.64
Chromosome chr14 | DEL threshold:  159.46 | INS threshold:  1071.55
Chromosome chr15 | DEL threshold:  194.90 | INS threshold:  1017.95

In [14]:
variants_merged = merge_variants(variants, binwidth)

  0%|          | 0/36846 [00:00<?, ?it/s]

In [15]:
create_vcf(variants_merged, output_vcf)